In [26]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
import re
from bs4 import BeautifulSoup
import contractions
import warnings
warnings.filterwarnings("ignore")
 

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/shreenidhihegde/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/shreenidhihegde/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
#! pip install bs4 # in case you don't have it installed

# Dataset: https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz

## Read Data

In [28]:
 """
 Here we are reading the data from tsv file as pandas dataframe.
 We use '\t' to seperate columns by a tab and "error_bad_lines = False" to drop bad lines from the DataFrame
 
 """  
text_data = pd.read_csv("data.tsv",error_bad_lines = False, sep = '\t')


b'Skipping line 16148: expected 15 fields, saw 22\nSkipping line 20100: expected 15 fields, saw 22\nSkipping line 45178: expected 15 fields, saw 22\nSkipping line 48700: expected 15 fields, saw 22\nSkipping line 63331: expected 15 fields, saw 22\n'
b'Skipping line 86053: expected 15 fields, saw 22\nSkipping line 88858: expected 15 fields, saw 22\nSkipping line 115017: expected 15 fields, saw 22\n'
b'Skipping line 137366: expected 15 fields, saw 22\nSkipping line 139110: expected 15 fields, saw 22\nSkipping line 165540: expected 15 fields, saw 22\nSkipping line 171813: expected 15 fields, saw 22\n'
b'Skipping line 203723: expected 15 fields, saw 22\nSkipping line 209366: expected 15 fields, saw 22\nSkipping line 211310: expected 15 fields, saw 22\nSkipping line 246351: expected 15 fields, saw 22\nSkipping line 252364: expected 15 fields, saw 22\n'
b'Skipping line 267003: expected 15 fields, saw 22\nSkipping line 268957: expected 15 fields, saw 22\nSkipping line 303336: expected 15 field

b'Skipping line 1968846: expected 15 fields, saw 22\nSkipping line 1999941: expected 15 fields, saw 22\nSkipping line 2001492: expected 15 fields, saw 22\nSkipping line 2011204: expected 15 fields, saw 22\nSkipping line 2025295: expected 15 fields, saw 22\n'
b'Skipping line 2041266: expected 15 fields, saw 22\nSkipping line 2073314: expected 15 fields, saw 22\nSkipping line 2080133: expected 15 fields, saw 22\nSkipping line 2088521: expected 15 fields, saw 22\n'
b'Skipping line 2103490: expected 15 fields, saw 22\nSkipping line 2115278: expected 15 fields, saw 22\nSkipping line 2153174: expected 15 fields, saw 22\nSkipping line 2161731: expected 15 fields, saw 22\n'
b'Skipping line 2165250: expected 15 fields, saw 22\nSkipping line 2175132: expected 15 fields, saw 22\nSkipping line 2206817: expected 15 fields, saw 22\nSkipping line 2215848: expected 15 fields, saw 22\nSkipping line 2223811: expected 15 fields, saw 22\n'
b'Skipping line 2257265: expected 15 fields, saw 22\nSkipping line

## Keep Reviews and Ratings

In [29]:
    #We are using dropna to look for missing values in the rows which has no review body and drop the corresponding rows.
    
    text_data.dropna(subset = ['review_body'], inplace= True)
    text_data.dropna(subset = ['star_rating'], inplace= True)
    
    
    # We Keep only the reviews and ratings of the initial data
    text_data = text_data[["star_rating","review_body"]]
    
    # We are including 3 sample reviews with the corresponding rating
    print ("\n -x--x- Three sample reviews with corresponding ratings -x--x- \n")
    print (text_data.sample(n=3, random_state=100))
    
    # We are reporting statistics of the ratings
    star_counts = text_data["star_rating"].value_counts()
    print("\n -x--x- Statistics of the ratings -x--x- \n")
    print(star_counts)
    
    


 -x--x- Three sample reviews with corresponding ratings -x--x- 

         star_rating                                        review_body
2264076          5.0  Do you know what's better than a Seattle Seaha...
2973343          4.0  the soup bowls are little bit smaller for nood...
2090625          3.0   They stain easy & handles get really hot to grab

 -x--x- Statistics of the ratings -x--x- 

5.0    3124595
4.0     731701
1.0     426870
3.0     349539
2.0     241939
Name: star_rating, dtype: int64


# Labelling Reviews:
## The reviews with rating 4,5 are labelled to be 1 and 1,2 are labelled as 0. Discard the reviews with rating 3'

In [30]:
# Using np.where we are putting conditions to label the ratings >=4 as 1 and ratins <=2 as 0. 
#The remainign ratings which is labelled as 3 will be labelled as -1.

text_data['label'] = np.where(text_data["star_rating"] >=4,1, np.where(text_data["star_rating"] <= 2,0,-1))
rating_count = text_data['label'].value_counts()

#We are getting the counts of all review labels before removing the reviews with rating 3
print("\n -x--x- Counts of all review labels before removing the reviews with rating 3 -x--x-\n")
print(rating_count)

#Discarding the reviews with rating 3
text_data = text_data[text_data["label"]!= -1]

#We are getting the counts of all review labels after removing the reviews with rating 3
print("\n -x--x- Counts of all review labels after removing the reviews with rating 3 -x--x- \n")
print(text_data['label'].value_counts())



 -x--x- Counts of all review labels before removing the reviews with rating 3 -x--x-

 1    3856296
 0     668809
-1     349539
Name: label, dtype: int64

 -x--x- Counts of all review labels after removing the reviews with rating 3 -x--x- 

1    3856296
0     668809
Name: label, dtype: int64


 ## We select 200000 reviews randomly with 100,000 positive and 100,000 negative reviews.



In [31]:
#We select sample of 100,000 positive and 100,000 negative reviews
negative = text_data.label[text_data.label.eq(0)].sample(100000).index
positive = text_data.label[text_data.label.eq(1)].sample(100000).index

#We combine both the selected samples
text_data = text_data.loc[negative.union(positive)]

# We are getting mean of each row based on characters in each row and then finding the mean of all the rows
review_mean = text_data.review_body.apply(lambda x : len(str(x))).mean()
# print 3 sample reviews

print(f'\n -x--x- The average length of the reviews in terms of character length in the dataset before cleaning is -x--x- \n{review_mean:.2f}')
print("\n -x--x- The sample reviews before cleaning -x--x- \n")
print(text_data["review_body"].sample(n=3, random_state=100))


                                          
                                                                                                                 


 -x--x- The average length of the reviews in terms of character length in the dataset before cleaning is -x--x- 
321.69

 -x--x- The sample reviews before cleaning -x--x- 

4466764    I was concerned about BPA and knew that using ...
230597     These koozies are great!  My boyfriend and I u...
1984586    Beautiful engineering but you spend the rest o...
Name: review_body, dtype: object


# Data Cleaning

## Convert the all reviews into the lower case.

In [32]:
#We use str.lower() to convert all the characters into lower characters
text_data["review_body"] = text_data["review_body"].str.lower()


 ## Remove the HTML and URLs from the reviews

In [33]:
#We use Beatiful soup to remove all the HTML Tags from the dataframe
text_data["review_body"] = text_data["review_body"].apply(lambda x: BeautifulSoup(str(x),"html.parser").get_text())

# We are here removing the URLs from the reviews
Url_pattern = r'\s*(https?://|www\.)+\S+(\s+|$)'
text_data["review_body"] = text_data["review_body"].apply(lambda x: re.sub(Url_pattern, " ", str(x), flags=re.UNICODE))



## remove non-alphabetical characters

In [34]:
#First we remove all the words starts with digits
text_data["review_body"] = text_data["review_body"].apply(lambda x: re.sub(r"[^\D']+", " ", str(x), flags=re.UNICODE))

#Next we remove all the words which starts with non alphabetic characters
text_data["review_body"] = text_data["review_body"].apply(lambda x: re.sub(r"[^\w']+", " ", str(x), flags=re.UNICODE))



## Remove the extra spaces between the words

In [35]:
#We remove the extra spaces from the dataset
# text_data["review_body"] = text_data["review_body"].replace('\s+', ' ', regex=True)
text_data["review_body"] = text_data["review_body"].apply(lambda x: re.sub(r"\s+", " ", str(x), flags=re.UNICODE))

## perform contractions on the reviews.

In [36]:
#We perform contractions using contractions.fix

def contractionfunction(i):
    i = i.apply(lambda x: contractions.fix(x))
    return i

text_data["review_body"] = contractionfunction(text_data["review_body"])
#We convert the characters into lowercase again as after contractions some words will get capitalized Ex: "i'm will" become "I am" after contraction.
text_data["review_body"] = text_data["review_body"].str.lower()


In [37]:
review_mean_new = text_data.review_body.apply(lambda x :len(str(x))).mean()
print("\n -x--x- The average length of the reviews in terms of character length in the dataset after cleaning is -x--x-\n", review_mean_new)
print("\n -x--x- The sample reviews after cleaning -x--x-\n")
print(text_data["review_body"].sample(n=3, random_state=100))


 -x--x- The average length of the reviews in terms of character length in the dataset after cleaning is -x--x-
 308.509475

 -x--x- The sample reviews after cleaning -x--x-

4466764    i was concerned about bpa and knew that using ...
230597     these koozies are great my boyfriend and i use...
1984586    beautiful engineering but you spend the rest o...
Name: review_body, dtype: object


# Pre-processing

## remove the stop words 

In [38]:
# We are removing stop words 

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

text_data["review_body"] = text_data["review_body"].apply(lambda x: " ".join([i for i in x.split() if i not in stop_words]))


## perform lemmatization  

In [39]:
#We perform lemmatization on the data

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
text_data["review_body"] = text_data["review_body"].apply(lambda x: " ".join(lemmatizer.lemmatize(i) for i in x.split()))



In [40]:
review_mean_3 = text_data.review_body.apply(lambda x : np.mean(len(str(x)))).mean()
print("\n -x--x- The average length of the reviews in terms of character length in the dataset after pre-processing is -x--x- \n", review_mean_3)
print("\n -x--x- The sample reviews after pre-processing -x--x-\n")
print(text_data["review_body"].sample(n=3, random_state=100))


 -x--x- The average length of the reviews in terms of character length in the dataset after pre-processing is -x--x- 
 188.91869

 -x--x- The sample reviews after pre-processing -x--x-

4466764    concerned bpa knew using reusing throw away bo...
230597     koozies great boyfriend used right getting rec...
1984586    beautiful engineering spend rest money buying ...
Name: review_body, dtype: object


# TF-IDF Feature Extraction

In [44]:
# We use train_test_split from sklearn to split the data into 80% training and 20% testing sets
from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import TfidfVectorizer

X_train, X_test, y_train, y_test = train_test_split(text_data["review_body"], text_data["label"], test_size=0.2, random_state=100)

#We ignore terms that have a document frequency strictly higher 0.7 and document frequency strictly lower than 1.
print(X_train.shape)

#vectorizer = TfidfVectorizer(min_df=1, max_df=0.7)
#Xtrain = vectorizer.fit_transform(X_train)
#Xtest = vectorizer.transform(X_test)

    

(160000,)


In [25]:
print(Xtrain[0])

  (0, 50261)	0.06901096342220804
  (0, 30367)	0.08581894523851961
  (0, 22092)	0.0572202415310822
  (0, 34060)	0.08789934045866332
  (0, 22266)	0.07387964188380705
  (0, 43499)	0.07670364997693523
  (0, 32171)	0.0342705606317413
  (0, 1928)	0.054736257249859206
  (0, 6263)	0.06485972126988566
  (0, 23208)	0.09838716528339621
  (0, 19726)	0.07716992316531952
  (0, 26331)	0.07166448130414782
  (0, 42784)	0.08066419777412423
  (0, 37153)	0.06727238461201715
  (0, 20162)	0.09569947342605921
  (0, 14720)	0.07169203441194372
  (0, 30269)	0.04740790084487304
  (0, 32866)	0.06724831097923202
  (0, 22352)	0.05553387400441643
  (0, 36681)	0.09931826965661106
  (0, 17250)	0.07273399589876606
  (0, 31052)	0.05059779551756077
  (0, 21115)	0.0555494367244654
  (0, 42204)	0.060463217989987404
  (0, 29455)	0.06426478989777605
  :	:
  (0, 8681)	0.1473710759325334
  (0, 38836)	0.09073054412759769
  (0, 32088)	0.0850062562639632
  (0, 7182)	0.513316707100233
  (0, 2481)	0.061211869802416316
  (0, 45977)	

In [17]:
# we standardize the data using StandardScaler from sklearn
from sklearn.preprocessing import StandardScaler

#Create the instance
sc = StandardScaler(with_mean=False)

#We fit the scaler to the training feauture set only
sc.fit(Xtrain)

#Scale or Transform the training and the testing tests using the scaler that was fitted to training data
Xtrain_std = sc.transform(Xtrain)
Xtest_std = sc.transform(Xtest)

# Perceptron

In [18]:
#implementation of perceptron
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Create a perceptron object with the parameters: 40 iterations (epochs) over the data, and a learning rate of 0.1
ppn = Perceptron(max_iter=40, eta0=0.1, random_state=100)

# Fit the model to the standardized data
ppn.fit(Xtrain_std, y_train)


# Apply the trained perceptron on the X data to make predicts for the Y test data
y_pred_test = ppn.predict(Xtest_std)

#We measure the performance using the "accuracy_score,f1_score,precision_score and recall_score"

print("\n -x--x- Accuracy, Precision, Recall, and f1-score on test data -x--x-\n")

print(f'Testing Accuracy:{accuracy_score(y_test, y_pred_test):.4f}')
print(f'Testing f1_Score:{f1_score(y_test, y_pred_test):.4f}')
print(f'Testing Precision:{precision_score(y_test, y_pred_test):.4f}')
print(f'Testing recall_score:{recall_score(y_test, y_pred_test):.4f}')

#Apply the trained perceptron on the data to make predicts for the trained data
y_pred_tarin = ppn.predict(Xtrain_std)


print("\n -x--x- Accuracy, Precision, Recall, and f1-score on train data -x--x-\n")

print(f'Training Accuracy:{accuracy_score(y_train, y_pred_tarin):.4f}')
print(f'Training f1_Score:{f1_score(y_train, y_pred_tarin):.4f}')
print(f'Training Precision:{precision_score(y_train, y_pred_tarin):.4f}')
print(f'Training recall_score:{recall_score(y_train, y_pred_tarin):.4f}')



 -x--x- Accuracy, Precision, Recall, and f1-score on test data -x--x-

Testing Accuracy:0.8279
Testing f1_Score:0.8274
Testing Precision:0.8267
Testing recall_score:0.8280

 -x--x- Accuracy, Precision, Recall, and f1-score on train data -x--x-

Training Accuracy:0.9262
Training f1_Score:0.9263
Training Precision:0.9266
Training recall_score:0.9259


# SVM

In [19]:
#implementation of SVM
from sklearn import svm

#Create a Classifier for svm
clf = svm.LinearSVC() # We are using Linear Kernel

#Train the model using the training sets
clf.fit(Xtrain, y_train)

#Apply the trained svm on Xtrain_std data to make predictions for the test data
y_pred_test = clf.predict(Xtest)


print("\n -x--x- Accuracy, Precision, Recall, and f1-score on test data -x--x- \n")

print(f'Testing Accuracy:{accuracy_score(y_test, y_pred_test):.4f}')
print(f'Testing f1_Score:{f1_score(y_test, y_pred_test):.4f}')
print(f'Testing Precision:{precision_score(y_test, y_pred_test):.4f}')
print(f'Testing recall_score:{recall_score(y_test, y_pred_test):.4f}')

#Apply the trained perceptron on the data to make predicts for the trained data
y_pred_tarin = clf.predict(Xtrain)

#We measure the performance using the "accuracy_score,f1_Score,Precision_score and recall_score"

print("\n -x--x- Accuracy, Precision, Recall, and f1-score on train data -x--x- \n")

print(f'Training Accuracy:{accuracy_score(y_train, y_pred_tarin):.4f}')
print(f'Training f1_Score:{f1_score(y_train, y_pred_tarin):.4f}')
print(f'Training Precision:{precision_score(y_train, y_pred_tarin):.4f}')
print(f'Training recall_score:{recall_score(y_train, y_pred_tarin):.4f}')


 -x--x- Accuracy, Precision, Recall, and f1-score on test data -x--x- 

Testing Accuracy:0.8977
Testing f1_Score:0.8970
Testing Precision:0.8992
Testing recall_score:0.8949

 -x--x- Accuracy, Precision, Recall, and f1-score on train data -x--x- 

Training Accuracy:0.9335
Training f1_Score:0.9334
Training Precision:0.9353
Training recall_score:0.9315


# Logistic Regression

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

# instantiate the model 
logreg = LogisticRegression(max_iter = 5000)

# fit the model with data
logreg.fit(Xtrain,y_train)

#Apply the trained Logistic Regression on Xtrain_std data to make predictions for the test data
y_pred_test=logreg.predict(Xtest)

#We measure the performance using the "accuracy_score,f1_Score,Precision_score and recall_score"
print("\n-x--x- Accuracy, Precision, Recall, and f1-score on test data --x--x-\n")

print(f'Testing Accuracy:{accuracy_score(y_test, y_pred_test):.4f}')
print(f'Testing f1_Score:{f1_score(y_test, y_pred_test):.4f}')
print(f'Testing Precision:{precision_score(y_test, y_pred_test):.4f}')
print(f'Testing recall_score:{recall_score(y_test, y_pred_test):.4f}')

#Apply the trained perceptron on the data to make predicts for the trained data
y_pred_tarin = logreg.predict(Xtrain)

#We measure the performance using the "accuracy_score"

print("\n-x--x- Accuracy, Precision, Recall, and f1-score on train data -x--x-\n")

print(f'Training Accuracy:{accuracy_score(y_train, y_pred_tarin):.4f}')
print(f'Training f1_Score:{f1_score(y_train, y_pred_tarin):.4f}')
print(f'Training Precision:{precision_score(y_train, y_pred_tarin):.4f}')
print(f'Training recall_score:{recall_score(y_train, y_pred_tarin):.4f}')





-x--x- Accuracy, Precision, Recall, and f1-score on test data --x--x-

Testing Accuracy:0.8982
Testing f1_Score:0.8973
Testing Precision:0.9021
Testing recall_score:0.8925

-x--x- Accuracy, Precision, Recall, and f1-score on train data -x--x-

Training Accuracy:0.9136
Training f1_Score:0.9133
Training Precision:0.9174
Training recall_score:0.9092


# Naive Bayes

In [21]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import MultinomialNB

#Create a Gaussian Classifier
model = MultinomialNB()

# Train the model using the training sets
model.fit(Xtrain, y_train)

#Predict Output
y_pred_test=model.predict(Xtest)

#We measure the performance using the "accuracy_score,f1_Score,Precision_score and recall_score"
print("\n -x--x- Accuracy, Precision, Recall, and f1-score on test data -x--x- \n")

print(f'Testing Accuracy:{accuracy_score(y_test, y_pred_test):.4f}')
print(f'Testing f1_Score:{f1_score(y_test, y_pred_test):.4f}')
print(f'Testing Precision:{precision_score(y_test, y_pred_test):.4f}')
print(f'Testing recall_score:{recall_score(y_test, y_pred_test):.4f}')

#Apply the trained perceptron on the data to make predicts for the trained data
y_pred_tarin = model.predict(Xtrain)

#We measure the performance using the "accuracy_score"

print("\n -x--x- Accuracy, Precision, Recall, and f1-score on train data -x--x- \n")

print(f'Training Accuracy:{accuracy_score(y_train, y_pred_tarin):.4f}')
print(f'Training f1_Score:{f1_score(y_train, y_pred_tarin):.4f}')
print(f'Training Precision:{precision_score(y_train, y_pred_tarin):.4f}')
print(f'Training recall_score:{recall_score(y_train, y_pred_tarin):.4f}')


 -x--x- Accuracy, Precision, Recall, and f1-score on test data -x--x- 

Testing Accuracy:0.8695
Testing f1_Score:0.8681
Testing Precision:0.8740
Testing recall_score:0.8622

 -x--x- Accuracy, Precision, Recall, and f1-score on train data -x--x- 

Training Accuracy:0.8866
Training f1_Score:0.8864
Training Precision:0.8891
Training recall_score:0.8836


In [ ]:
#cleaning data

def clean_text(
    string: str, 
    punctuations=r'''!()-[]{};:'"\,<>./?@#$%^&*_~''',
    stop_words=['the', 'a', 'and', 'is', 'be', 'will']) -> str:
    """
    A method to clean text 
    """
    # Cleaning the urls
    string = re.sub(r'https?://\S+|www\.\S+', '', string)

    # Cleaning the html elements
    string = re.sub(r'<.*?>', '', string)

    # Removing the punctuations
    for x in string.lower(): 
        if x in punctuations: 
            string = string.replace(x, "") 

    # Converting the text to lower
    string = string.lower()

    # Removing stop words
    string = ' '.join([word for word in string.split() if word not in stop_words])

    # Cleaning the whitespaces
    string = re.sub(r'\s+', ' ', string).strip()

    return string        